### Python imports

In [1]:
import xarray as xr
import os
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import sleep
from functools import partial
import time
import numcodecs
import zarr
import dask
from dask.distributed import as_completed

### Inputs

In [2]:
import io
import boto3
def read_dataset_inmemory(s3_path: str) -> xr.Dataset:
    """Read a NetCDF as an XArray using in-memory data"""
    try:
        with io.BytesIO() as inmemoryfile:
            # Use boto to download a file to memory
            session = boto3.Session(profile_name='nonproduction-admin')
            s3 = session.client('s3')
            bucket, key = s3_path.replace("s3://", "").split("/", 1)
            s3.download_fileobj(bucket, key, inmemoryfile)
            inmemoryfile.seek(0)
            return xr.open_dataset(inmemoryfile)
    except ValueError as e:
        print(f"Failed to open the file with error: {e}")
        return None

### Argo Processor

**Source notebook**
https://medium.com/@nicolasmortimer/argo-floats-zarr-and-pangeo-d74fc6d4ce35

*Written by Nicolas Mortimer*

In [3]:
data_types ={'CONFIG_MISSION_NUMBER':'float32','CYCLE_NUMBER':'float32','DATA_CENTRE':'|U2','DATA_MODE':'|U1',
             'DATA_STATE_INDICATOR':'|U4','DC_REFERENCE':'|U32','DIRECTION':'|U1','FIRMWARE_VERSION':'|U32',
             'FLOAT_SERIAL_NO':'|U32','JULD':'float32','JULD_LOCATION':'float32','JULD_QC':'|U1','LATITUDE':'float32',
             'LONGITUDE':'float32','PI_NAME':'|U64','PLATFORM_NUMBER':'|U8','PLATFORM_TYPE':'|U32','POSITIONING_SYSTEM':'|U8',
             'POSITION_QC':'|U1','PRES':'float32','PRES_ADJUSTED':'float32','PRES_ADJUSTED_ERROR':'float32',
             'PRES_ADJUSTED_QC':'|U1','PRES_QC':'|U1','PROFILE_PRES_QC':'|U1','PROFILE_PSAL_QC':'|U1','PROFILE_TEMP_QC':'|U1',
             'PROJECT_NAME':'|U64','PSAL':'float32','PSAL_ADJUSTED':'float32','PSAL_ADJUSTED_ERROR':'float32',
             'PSAL_ADJUSTED_QC':'|U1','PSAL_QC':'|U1','TEMP':'float32','TEMP_ADJUSTED':'float32','TEMP_ADJUSTED_ERROR':'float32',
             'TEMP_ADJUSTED_QC':'|U1','TEMP_QC':'|U1','VERTICAL_SAMPLING_SCHEME':'|U256','WMO_INST_TYPE':'|U4'}

data_levels =['PRES','PRES_ADJUSTED','PRES_ADJUSTED_ERROR','PRES_ADJUSTED_QC','PRES_QC','PSAL','PSAL_ADJUSTED',
              'PSAL_ADJUSTED_ERROR','PSAL_ADJUSTED_QC','PSAL_QC','TEMP','TEMP_ADJUSTED','TEMP_ADJUSTED_ERROR',
              'TEMP_ADJUSTED_QC','TEMP_QC']

def process_mf(dsinput,levels,data_types=data_types,data_levels=data_levels):
    ds = xr.Dataset()
    dims =('N_PROF','N_LEVELS')
    # The number of profiles is indicated by the N_PROF dimension
    # The number of pressure levels is indicated by the N_LEVELS dimension
    pading =xr.DataArray(np.ones((len(dsinput.N_PROF),levels-len( dsinput.N_LEVELS))) *np.nan,dims=dims)
    pad_qc = xr.DataArray(np.chararray((len(dsinput.N_PROF),levels-len( dsinput.N_LEVELS))),dims=dims)
    pad_qc[:] = b' '
    for varname in data_types.keys():
        if varname in dsinput.data_vars:
            da = dsinput[varname]
            if 'N_LEVELS' in da.dims:   
                if varname in dsinput.data_vars:
                    if varname.endswith('QC'):
                        da = xr.concat([dsinput[varname],pad_qc],dim='N_LEVELS').astype(data_types[varname])
                    else:
                        da = xr.concat([dsinput[varname],pading],dim='N_LEVELS').astype(data_types[varname])
            else:
                da = dsinput[varname].astype(data_types[varname])
        else:
            if varname in data_levels:
                if data_types[varname]=='float32':
                    da = xr.DataArray(np.ones((len(dsinput.N_PROF),levels), dtype='float32')*np.nan , name=varname, dims=['N_PROF','N_LEVELS'])
                else:
                    p=np.chararray((len(dsinput.N_PROF),levels))
                    p[:]=b'0'
                    da = xr.DataArray(p.astype(data_types[varname]), name=varname, dims=['N_PROF','N_LEVELS'])
            else:
                if data_types[varname]=='float32':
                    da = xr.DataArray(np.ones(len(dsinput.N_PROF), dtype="float32")*np.nan , name=varname, dims=['N_PROF'])
                else:
                    p=np.chararray((len(dsinput.N_PROF)))
                    p[:]=b'0'
                    da = xr.DataArray(p.astype(data_types[varname]), name=varname, dims=['N_PROF'])
        if not ('HISTORY' in varname) and ('N_CALIB' not in da.dims) and ('N_PARAM' not in da.dims) and  ('N_PROF' in da.dims):
                ds[varname]= da
    return ds.chunk({'N_LEVELS':levels})

In [4]:
@dask.delayed
def process_float(s3_uri):
    preproc = partial(process_mf,levels=3000)
    file = read_dataset_inmemory(s3_uri)
    data = preproc(file)
    return data

In [5]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

### Generate dataset and export to Zarr

In [6]:
%%time
import s3fs
emr_dns = 'localhost'
source_path = 's3://imos-data/IMOS/Argo/dac/csiro/1901119/profiles/*.nc'
# store_path = './emr/argo.zarr/'
store_path = 's3://imos-data-pixeldrill/vhnguyen/emr/argo/temp/temp.zarr'
aws_profile = 'nonproduction-admin'
s3 = s3fs.S3FileSystem(profile=aws_profile, anon=False)
glob_result = s3.glob(source_path)
input_paths = []
input_paths.extend(['s3://' + path for path in glob_result])

CPU times: user 158 ms, sys: 25.6 ms, total: 183 ms
Wall time: 2.09 s


In [7]:
input_paths[0]

's3://imos-data/IMOS/Argo/dac/csiro/1901119/profiles/D1901119_001.nc'

In [8]:
read_dataset_inmemory(input_paths[0])

<xarray.Dataset>
Dimensions:                       (N_PROF: 1, N_PARAM: 3, N_LEVELS: 70,
                                   N_CALIB: 1, N_HISTORY: 9)
Dimensions without coordinates: N_PROF, N_PARAM, N_LEVELS, N_CALIB, N_HISTORY
Data variables: (12/64)
    DATA_TYPE                     object ...
    FORMAT_VERSION                object ...
    HANDBOOK_VERSION              object ...
    REFERENCE_DATE_TIME           object ...
    DATE_CREATION                 object ...
    DATE_UPDATE                   object ...
    ...                            ...
    HISTORY_ACTION                (N_HISTORY, N_PROF) object ...
    HISTORY_PARAMETER             (N_HISTORY, N_PROF) object ...
    HISTORY_START_PRES            (N_HISTORY, N_PROF) float32 ...
    HISTORY_STOP_PRES             (N_HISTORY, N_PROF) float32 ...
    HISTORY_PREVIOUS_VALUE        (N_HISTORY, N_PROF) float32 ...
    HISTORY_QCTEST                (N_HISTORY, N_PROF) object ...
Attributes:
    title:                Argo float vertical profile
    institution:          CSIRO
    source:               Argo float
    history:              2013-07-27T12:59:35Z creation;2014-08-15T15:22:01Z ...
    references:           http://www.argodatamgt.org/Documentation
    user_manual_version:  3.1
    Conventions:          Argo-3.1 CF-1.6
    featureType:          trajectoryProfile

### Using Dask

#### Fargate cluster

In [9]:
# from dask_cloudprovider.aws import FargateCluster
# cluster = FargateCluster(image="ghcr.io/vietnguyengit/vietnguyen-dask:main", scheduler_timeout="60 minutes", task_role_arn="arn:aws:iam::615645230945:role/ManualDaskZarrCreation",
#                          scheduler_cpu=4096, scheduler_mem=30720, n_workers=10)

In [10]:
# from dask.distributed import Client
# client = Client(cluster)
# display(client)

#### Local cluster

In [11]:
# import dask
# from dask.distributed import Client, LocalCluster
# with dask.config.set({'temporary_directory': '/home/ec2-user/dask/'}):
#     # set up cluster and workers
#     cluster = LocalCluster(n_workers=4, memory_limit='8GB', processes=True, 
#                            threads_per_worker=4, dashboard_address=':0', ip=emr_dns)
#     client = Client(address=cluster.scheduler_address)

# print(f'http://{emr_dns}'+':{port}/status'.format(port=client.scheduler_info().get('services').get('dashboard')))

#### Dask extension local cluster

In [12]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:44733")
client

<Client: 'tcp://127.0.0.1:44733' processes=4 threads=16, memory=31.10 GiB>

In [ ]:
store= s3fs.S3Map(root=f'{store_path}', s3=s3, check=False)
start_time = time.time()

input_paths = input_paths[:8]
all_chunked_paths = list(chunks(input_paths, 2))

print('************** Initilising Futures **************')
futures = []
for i in tqdm(range(len(all_chunked_paths))):
    datasets = []
    for path in all_chunked_paths[i]:
        # datasets.append(process_float(path))
        temp = process_float(path)
        datasets.append(temp.persist())
    zarrs = dask.compute(*datasets)   
    zarrs_remote = client.scatter(zarrs) #send zarrs to cluster
    ds_future = [client.submit(xr.concat, zarrs_remote, 
                               dim='N_PROF', coords='minimal',
                               compat='override',combine_attrs='override', 
                               fill_value='')]
    for done in as_completed(ds_future):
        futures.append(done)

print('************** Processing Zarr **************')
for i in tqdm(range(len(futures))):
    overwrite = True if i == 0 else False
    ds = futures[i].result()
    for var in ds.data_vars:
        ds[var].encoding = {} 
    if overwrite:
        # to_zarr() lazily
        z = ds.to_zarr(store, mode='w', consolidated=True, compute=False)
    else:
        z = ds.to_zarr(store, mode='a', append_dim='N_PROF', consolidated=True, compute=False)
    z_persisted = z.persist()
    z_persisted.compute()

print('*********************************************')
print("---------- Total: %.2f seconds ----------" % (time.time() - start_time))
print('*********************************************')

************** Initilising Futures **************


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.85s/it]


************** Processing Zarr **************


 75%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                          | 3/4 [04:57<01:28, 88.92s/it]

In [ ]:
# input_paths = input_paths[:8]
# store= s3fs.S3Map(root=f'{store_path}', s3=s3, check=False)
# start_time = time.time()
# def process_float(s3_uri):
#     preproc = partial(process_mf,levels=3000)
#     file = read_dataset_inmemory(s3_uri)
#     data = preproc(file)
#     return data
# print('************** Initilising Futures **************')
# futures = []
# for i in tqdm(range(len(input_paths))):
#     futures.append(client.submit(process_float, input_paths[i], retries=20))

# # Break the futures list into smaller chunks
# all_chunked_futures = list(chunks(futures, 2))
# print('************** Processing Zarr **************')
# for i in tqdm(range(len(all_chunked_futures))):
#     overwrite = True if i == 0 else False
#     zarrs = client.gather(all_chunked_futures[i]) # result return to local from the cluster
#     ds = xr.concat(zarrs, dim='N_PROF', coords='minimal',compat='override',combine_attrs='override', fill_value='')
#     # chunked = ds.chunk(chunks=1000)
#     for var in ds.data_vars:
#         ds[var].encoding = {}
#     if overwrite:
#         z = ds.to_zarr(store, mode='w', consolidated=True, compute=False) # return delayed obj
#     else:
#         z = ds.to_zarr(store, mode='a', append_dim='N_PROF', consolidated=True, compute=False)
#     z_persisted = z.persist()
#     z_persisted.compute()
    
# print('*********************************************')
# print("---------- Total: %.2f seconds ----------" % (time.time() - start_time))
# print('*********************************************')

### Open Zarr

In [ ]:
%%time
data = xr.open_zarr(store_path)
data

In [ ]:
%%time
import matplotlib.pyplot as plt
from datetime import datetime
nprof = 200
def np_dt64_to_dt(in_datetime: np.datetime64) -> str:
    """Convert numpy datetime64 to datetime"""
    dt = datetime.fromtimestamp(in_datetime.astype(int) / 1e9)
    return dt.strftime("%Y-%m-%dT%H:%M:%SZ")

plt.figure(figsize=(8,6))

nprof = round(len(input_paths) / 2) #Selected profile
plt.scatter(data.PSAL_ADJUSTED[nprof], data.TEMP_ADJUSTED[nprof], c=data.PRES_ADJUSTED[nprof], cmap='viridis_r')
plt.xlabel('Salinity');
plt.ylabel('Temperature (°C)')

cbh = plt.colorbar();
cbh.set_label('Pressure (dbar)')

plt.grid()
plt.title('Argo Float #%d on %s' % (data.PLATFORM_NUMBER[nprof].values, np_dt64_to_dt(data.JULD[nprof].values)), fontweight='bold');
plt.show()

In [ ]:
# cluster.close()
# client.close()